In [2]:
import pandas as pd
import numpy as np
from numpy import log2 as log
error_handler = np.finfo(float).eps

In [51]:
import os
file_path = r'C:\Users\hp\CSE 3210\Lab 2\vehicle2.csv'
file_name = os.path.basename(file_path)
name = (os.path.basename(file_path).split(".")[0])
file_name

'vehicle2.csv'

In [52]:
data = pd.read_csv(file_name)
data

,95,48,83,178,72,10,162,42,20,159,176,379,184,70,6,16,187,197,negative
0,91,41,84,141,57,9,149,45,19,143,170,330,158,72,9,14,189,199,negative
1,104,50,106,209,66,10,207,32,23,158,223,635,220,73,14,9,188,196,negative
2,93,41,82,159,63,9,144,46,19,143,160,309,127,63,6,10,199,207,negative
3,85,44,70,205,103,52,149,45,19,144,241,325,188,127,9,11,180,183,positive
4,107,57,106,172,50,6,255,26,28,169,280,957,264,85,5,9,181,183,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
840,93,39,87,183,64,8,169,40,20,134,200,422,149,72,7,25,188,195,negative
841,89,46,84,163,66,11,159,43,20,159,173,368,176,72,1,20,186,197,negative
842,106,54,101,222,67,12,222,30,25,173,228,721,200,70,3,4,187,201,negative
843,86,36,78,146,58,7,135,50,18,124,155,270,148,66,0,25,190,195,negative


Entropy of Class

In [53]:
def class_entropy(data):
    en_class = 0
    className = data.columns[-1] # last column is class
    patterns = data[className].unique() # yes or no
    for pattern in patterns:
        quotient = data[className].value_counts()[pattern] / data[className].shape[0] # probability
        en_class += -quotient*log(quotient) # equation to determine entropy

    return en_class 

In [54]:
class_entropy(data)

1.4899894050657077

Entropy of Features

In [55]:
def feature_entropy(data, feat):
    className = data.columns[-1] # last column is class
    patterns = data[className].unique() # yes or no
    instances = data[feat].unique() # unique value of features
    en_feat = 0
    for ins in instances:
        en_each = 0
        for pattern in patterns:
            nr = len(data[feat][data[feat] == ins][data[className] == pattern])  # matching with class
            dr = len(data[feat][data[feat] == ins])
            quotient = nr/(dr + error_handler)
            en_each += -quotient*log(quotient + error_handler)   # entropy equation
        q2 = dr / data.shape[0]
        en_feat += -q2 * en_each
    return abs(en_feat)

In [56]:
feature_entropy(data, 'Age')

KeyError: 'Age'

In [57]:
def root_node(data):
    feat_entropy = []
    ig = []
    for feat in data.columns[:-1]:
        ig.append(class_entropy(data) - feature_entropy(data, feat))  # information gain calculation
    return data.columns[:-1][np.argmax(ig)]   # find max information gain

In [58]:
root_node(data)

' 379'

In [59]:
def generate_subtable(data, root, value):
    return data[data[root] == value].reset_index(drop = True)   # create subtable with resetting index

In [60]:
generate_subtable(data, 'Age', '>55')

KeyError: 'Age'

In [61]:
def tree_Building(data, tree = None):
    className = data.columns[-1]  # last column is class
    
    root = root_node(data) # getting root node
    
    featValue = np.unique(data[root]) # unique values of root node
    
    if tree is None: # create an empty tree
        tree = {}
        tree[root] = {}
        
    for value in featValue:
        subtable = generate_subtable(data, root, value)  # for each unique value create subtables
        col, counts = np.unique(subtable[className], return_counts = True) # Checking which attribute matches with class
        
        if len(counts) == 1:   # if there is only one such attribute that matches with class then that is the node
            tree[root][value] = col[0]
        else:
            tree[root][value] = tree_Building(subtable) # else explore that node
            
    return tree

In [ ]:
tree = tree_Building(data)
import pprint
pprint.pprint(tree)

In [ ]:
import pydot

def draw(parent, child):
    edge = pydot.Edge(parent, child)
    graph.add_edge(edge)

def visit(dic, parent = None):
    for key, value in dic.items():
        if isinstance(value, dict): # if value is a dictionary
            if parent:  # if parent node
                draw(parent, str(key)) # draw an edge from parent node to feature
            visit(value, str(key)) # recursively create subtree
        else:
            draw(parent, str(key)) # if value is not a dictionary
            draw(str(key), value)  # then draw a edge

graph = pydot.Dot(graph_type='graph')
visit(tree)
graph.write_png(name + '.png')